# Import Section
---

In [1]:
"""
Easy training, quantization TFLite testing and deployment with jupyter and UI
"""
import shutil
import os
from pathlib import Path

import ipywidgets as widgets
from ipywidgets import interact_manual, Layout, Box, Textarea, Label
from IPython.display import display, HTML, clear_output
from ipyfilechooser import FileChooser

# Widgets Control Section
---

In [2]:
class TrainConfigAndCmdsWidgets:
    """
    A class to manage the widgets for configuring training parameters and commands.
    This likely forms part of a larger machine learning or deep learning application.
    """

    def __init__(self):

        self.cfg_dir = "cfg"
        self.my_cfg_path = Path(self.cfg_dir) / "my_config.yaml"
        self.m_src_tflite = None

        self.tflite_file_loc = None
        self.src_audio_test_file = None

        form_item_layout = Layout(
            display="flex",
            flex_flow="row",
            justify_content="space-between",
        )

        # train
        self.a_ta = widgets.Text(value="yolox_n_nu_320", placeholder="Type something", disabled=False)
        self.e_ta = widgets.BoundedIntText(value=320, min=96, max=416, step=1, disabled=False)
        self.b_ta = widgets.IntSlider(value=64, min=4, max=128, step=4)
        self.c_ta = widgets.BoundedIntText(value=100, min=5, max=10000, step=1, disabled=False)
        self.d_ta = widgets.BoundedIntText(value=6, min=1, max=1000, step=1, disabled=False)
        self.j_ta = Textarea(value="datasets/medicinev2_coco", disabled=False)
        self.k_ta = Textarea(value="medicinev2_train.json", disabled=False)
        self.l_ta = Textarea(value="medicinev2_val.json", disabled=False)
        self.f_ta = widgets.Checkbox(value=True, disabled=False, indent=False)
        self.g_ta = Textarea(value="pretrain/tflite_yolox_nano_ti/320_DW/yolox_nano_320_DW_ti_lite.pth", disabled=False)

        form_train_items = [
            Box([Label(value="Project Name"), self.a_ta], layout=form_item_layout),
            Box([Label(value="Image Size (WxH)"), self.e_ta], layout=form_item_layout),
            Box([Label(value="Batch Size"), self.b_ta], layout=form_item_layout),
            Box([Label(value="Epochs"), self.c_ta], layout=form_item_layout),
            Box([Label(value="Number of Classes"), self.d_ta], layout=form_item_layout),
            Box([Label(value="Dataset Dir"), self.j_ta], layout=form_item_layout),
            Box([Label(value="Train Annotation File"), self.k_ta], layout=form_item_layout),
            Box([Label(value="Validation Annotation File"), self.l_ta], layout=form_item_layout),
            Box([Label(value="Use Pretrain Model"), self.f_ta], layout=form_item_layout),
            Box([Label(value="Pretrain Model File"), self.g_ta], layout=form_item_layout),
        ]

        self.form_output_train_cmd = Box(
            form_train_items,
            layout=Layout(
                display="flex",
                flex_flow="column",
                border="solid 1px lightblue",
                align_items="stretch",
                width="100%",
            ),
        )

        # run buttoms box
        self.a_rb = widgets.Checkbox(value=True, disabled=False, indent=False)
        self.b_rb = widgets.Button(description="Run", layout=Layout(width="30%", height="30px"), button_style="success")

        form_run_buttoms_items = [
            Box([Label(value="Directly Run on Jupyter"), self.a_rb], layout=form_item_layout),
            Box([Label(value="Start to Train"), self.b_rb], layout=form_item_layout),
        ]

        self.form_run_buttoms_cmd = Box(
            form_run_buttoms_items,
            layout=Layout(
                display="flex",
                flex_flow="column",
                border="solid 1px lightblue",
                align_items="stretch",
                width="100%",
            ),
        )

        # test
        self.a_tt = widgets.Button(description="Setting", layout=Layout(width="30%", height="30px"), button_style="success")
        self.c_tt = widgets.Button(description="Run", layout=Layout(width="30%", height="30px"), button_style="success")
        self.b_tt = Textarea(value="datasets/medicinev2_coco", disabled=False)
        self.d_tt = Textarea(value="medicinev2_val.json", disabled=False)

        form_test_items = [
            Box([Label(value="Choose the tflite file"), self.a_tt], layout=form_item_layout),
            Box([Label(value="Dataset Dir"), self.b_tt], layout=form_item_layout),
            Box([Label(value="Test Annotation File"), self.d_tt], layout=form_item_layout),
            Box([Label(value="Start to Test"), self.c_tt], layout=form_item_layout),
        ]

        self.form_output_test_cmd = Box(
            form_test_items,
            layout=Layout(
                display="flex",
                flex_flow="column",
                border="solid 3px lightgreen",
                align_items="stretch",
                width="50%",
            ),
        )

        # convert model cpp
        self.a_cm = widgets.Text(value=r"YOLOX_outputs/yolox_n_nu_320", placeholder="Type something", disabled=False)
        self.b_cm = widgets.Text(value=r"datasets/coco/train2017", placeholder="Type something", disabled=False)
        self.c_cm = widgets.Text(value="yolox_n_nu_320_mytask", placeholder="Type something", disabled=False)
        self.d_cm = widgets.Button(description="Gen Bat file", layout=Layout(width="30%", height="30px"), button_style="success")

        form_convert_items_paths = [
            Box([Label(value="Project Dir"), self.a_cm], layout=form_item_layout),
            Box([Label(value="Train Dataset Dir"), self.b_cm], layout=form_item_layout),
            Box([Label(value="Your Modle File Name"), self.c_cm], layout=form_item_layout),
        ]

        form_convert_items = [
            Box(form_convert_items_paths, layout=Layout(display="flex", flex_flow="column", justify_content="center", border="dotted 3px lightblue", align_items="stretch", width="70%")),
            Box([Label(value="Convert to tflite int8 & Vela"), self.d_cm], layout=form_item_layout),
        ]

        self.form_output_convert_cmd = Box(
            form_convert_items,
            layout=Layout(
                display="flex",
                flex_flow="column",
                justify_content="center",
                border="solid 3px lightgreen",
                align_items="stretch",
                width="70%",
            ),
        )

    def move_allfiles(self, src_folder, dst_folder):
        """
        Moves all files from a source directory to a destination directory.
        Directories within the source directory are not moved; only their contents.
        """

        files = os.listdir(src_folder)
        for f in files:
            fullpath = os.path.join(src_folder, f)
            if os.path.isdir(fullpath):  # copy whole folder
                shutil.move(fullpath, dst_folder)
                print(f"Copy finish: {f}")

    def show_headline(self, output):
        """
        show headline
        """
        html0 = widgets.HTML(value=f"<b><font color='lightblue'><font size=4>{output}</b>")
        display(html0)

    def show_main(self):
        """
        show main
        """
        intro_text = "Yolox-nano-nu Easy Setting"
        html_widget = widgets.HTML(value=f"<b><font color='lightgreen'><font size=6>{intro_text}</b>")
        display(html_widget)

        # Create an accordion and put the 2 boxes
        accordion = widgets.Accordion(children=[self.form_output_train_cmd]).add_class("parentstyle")
        display(HTML("<style>.parentstyle > .p-Accordion-child > .p-Collapse-header{background-color:green}</style>"))
        accordion.set_title(0, "Configure the Training")

        # Create a box combining with 2 elements
        box_data_train = Box(
            [accordion, self.form_run_buttoms_cmd],
            layout=Layout(
                display="flex",
                flex_flow="column",
                border="solid 3px lightgreen",
                align_items="stretch",
                width="50%",
            ),
        )

        # Create a tab and put the 2 boxes
        tab = widgets.Tab(children=[box_data_train, self.form_output_test_cmd, self.form_output_convert_cmd]).add_class("parentstyle")
        tab_contents = ["Train", "Test", "Deployment"]
        tab.titles = tab_contents

        output_widgets = widgets.Output(layout=Layout(border="1px solid green"))

        def act_para(*, a_ta, b_ta, c_ta, d_ta, e_ta, j_ta, k_ta, l_ta, a_cm, b_cm, c_cm, a_rb):

            # If any value is changed, clear the widgets
            with output_widgets:
                output_widgets.clear_output()

        # ------------------#
        # widgets.Accordion's interactive input with action function `act_para()`
        # ------------------#
        out_inter = widgets.interactive_output(
            act_para,
            {
                "a_ta": self.a_ta,
                "b_ta": self.b_ta,
                "c_ta": self.c_ta,
                "d_ta": self.d_ta,
                "e_ta": self.e_ta,
                "j_ta": self.j_ta,
                "k_ta": self.k_ta,
                "l_ta": self.l_ta,
                "a_cm": self.a_cm,
                "b_cm": self.b_cm,
                "c_cm": self.c_cm,
                "a_rb": self.a_rb,
            },
        )

        display(tab, out_inter)

        # ------------------#
        # for labelimg cmd, move to outside of act_para to prevent keep trigering
        # ------------------#
        # output_widgets = widgets.Output(layout=Layout(border = '1px solid green'))
        display(output_widgets)

        def on_button_clicked_train(b):
            with output_widgets:
                clear_output()
                # print("Train. . .")
                self.run_train()

        self.b_rb.on_click(on_button_clicked_train)

        def on_button_clicked_cpp(b):
            with output_widgets:
                clear_output()
                self.convert_tflu()

        self.d_cm.on_click(on_button_clicked_cpp)

        def on_button_clicked_choose_tflite_test(b):
            with output_widgets:
                clear_output()
                self.choose_tflite()

        self.a_tt.on_click(on_button_clicked_choose_tflite_test)

        def on_button_clicked_test(b):
            with output_widgets:
                clear_output()
                self.run_test_tflite()

        self.c_tt.on_click(on_button_clicked_test)

    def choose_tflite(self):
        """
        choose tflite
        """
        path_ftflite = os.path.join(os.getcwd(), "YOLOX_outputs")
        f_tflite = FileChooser(path_ftflite)
        # Restrict navigation to /Users
        f_tflite.sandbox_path = os.getcwd()
        f_tflite.filter_pattern = ["*.tflite"]
        f_tflite.title = "<b><font color='lightblue'><font size=4>Choose the Tflite for test single audio.</b>"
        display(f_tflite)

        def act_test():
            work_dir_name = os.getcwd().split("\\")[-1]
            m_src_dir = f_tflite.selected_path.split(work_dir_name)[-1]
            m_src_tflite = f_tflite.selected.split("\\")[-1]
            print(f"The chosen dir: {m_src_dir}")
            print(f"The chosen tflite: {m_src_tflite}")
            self.m_src_tflite = os.path.join(m_src_dir, m_src_tflite)
            print("Finish!")

        evt = interact_manual(act_test)
        evt.widget.children[0].description = "Choose this tflite file"  # because there are 3 parameter of the evt
        evt.widget.children[0].button_style = "primary"

    def run_train(self):
        """
        training, 2 choices, one for showing cmd, the other is trainig in Jupyter.
        """

        if self.a_rb.value:
            if self.f_ta.value:
                %run tools/train.py -f exps/default/yolox_nano_ti_lite_nu.py -d 1 -b $self.b_ta.value --fp16 -o -c $self.g_ta.value -ename $self.a_ta.value -im $self.e_ta.value -epo $self.c_ta.value -nc $self.d_ta.value --data_dir $self.j_ta.value --train_ann $self.k_ta.value --val_ann $self.l_ta.value
            else:
                %run tools/train.py -f exps/default/yolox_nano_ti_lite_nu.py -d 1 -b $self.b_ta.value --fp16 -o -ename $self.a_ta.value -im $self.e_ta.value -epo $self.c_ta.value -nc $self.d_ta.value --data_dir $self.j_ta.value --train_ann $self.k_ta.value --val_ann $self.l_ta.value
        else:
            # get the training cmd
            self.show_headline("Please copy these commands to the CMD.exe Prompt to execute. ")

            self.show_headline("(1.) Please excute the below commands under the working directory")
            print(f"cd {os.path.join(os.getcwd())}")
            self.show_headline("(2.) Train:")
            if self.f_ta.value:
                train_cmd = (
                    r"python tools/train.py -f exps/default/yolox_nano_ti_lite_nu.py -d 1 -b {} --fp16 -o -c {} -ename {} -im {} -epo {} -nc {} --data_dir {} --train_ann {} --val_ann {}".format(
                        self.b_ta.value, self.g_ta.value, self.a_ta.value, self.e_ta.value, self.c_ta.value, self.d_ta.value, self.j_ta.value, self.k_ta.value, self.l_ta.value
                    )
                )
            else:
                train_cmd = r"python tools/train.py -f exps/default/yolox_nano_ti_lite_nu.py -d 1 -b {} --fp16 -o -ename {} -im {} -epo {} -nc {} --data_dir {} --train_ann {} --val_ann {}".format(
                    self.b_ta.value, self.a_ta.value, self.e_ta.value, self.c_ta.value, self.d_ta.value, self.j_ta.value, self.k_ta.value, self.l_ta.value
                )
            print(train_cmd)

    def run_test_tflite(self):
        """
        test tflite
        """

        print(f"The tflite file: {self.m_src_tflite}")

        %run demo/TFLite/tflite_eval.py -m $self.m_src_tflite -v $self.b_tt.value -a $self.d_tt.value

        print("Finish !!")

    def update_line_in_file(self, file_path, search_phrase, new_value):
        """
        update line in batch file for updating parameters in *.bat
        """
        with open(file_path, "r", encoding="utf-8") as file:
            lines = file.readlines()

        with open(file_path, "w", encoding="utf-8") as file:
            for line in lines:
                if line.startswith(search_phrase[0]):
                    file.write(f"{new_value[0]}\n")
                elif line.startswith(search_phrase[1]):
                    file.write(f"{new_value[1]}\n")
                elif line.startswith(search_phrase[2]):
                    file.write(f"{new_value[2]}\n")
                elif line.startswith(search_phrase[3]):
                    file.write(f"{new_value[3]}\n")
                elif line.startswith(search_phrase[4]):
                    file.write(f"{new_value[4]}\n")
                elif line.startswith(search_phrase[5]):
                    file.write(f"{new_value[5]}\n")
                elif line.startswith(search_phrase[6]):
                    file.write(f"{new_value[6]}\n")
                else:
                    file.write(line)

    def convert_tflu(self):
        """
        convert to tflu
        """
        print("Update the variables of yolox_convert.bat done!")

        file_path = "yolox_convert.bat"
        search_phrase = ["set IMG_SIZE", "set NUM_CLS", "set MODEL_FILE_NAME", "set OUTPUT_DIR", "set TRAIN_DATASET", "set IMAGE_SIZE_EXPR", "set CLASS_EXPR"]
        image_size_update = f'set IMAGE_SIZE_EXPR="extern const int originalImageSize = {self.e_ta.value}"'
        class_expr_update = f'set CLASS_EXPR="extern const int numClasses = {self.d_ta.value}"'
        new_value = [
            f"set IMG_SIZE={self.e_ta.value}",
            f"set NUM_CLS={self.d_ta.value}",
            f"set MODEL_FILE_NAME={self.c_cm.value}",
            f"set OUTPUT_DIR={self.a_cm.value}",
            f"set TRAIN_DATASET={self.b_cm.value}",
            image_size_update,
            class_expr_update,
        ]
        self.update_line_in_file(file_path, search_phrase, new_value)

        self.show_headline("Please execute yolox_convert.bat in current work directory.")

# Run Section
---

In [3]:
act = TrainConfigAndCmdsWidgets()
act.show_main()

HTML(value="<b><font color='lightgreen'><font size=6>Yolox-nano-nu Easy Setting</b>")

Output()

Output(layout=Layout(border_bottom='1px solid green', border_left='1px solid green', border_right='1px solid g…